In [ ]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import row_number, dense_rank, col, mean, lit
import itertools
import pandas as pd
import numpy as np

In [ ]:
main_data = sqlContext.sql(""" select * from dnd.cftex_v2 """)
data = main_data.toPandas()
data = data.dropna()
data = data[(data['CONTAINER_TYPE_CD'] == 'DRY')&((data['CONTAINER_SIZE_CD'] == '20') | (data['CONTAINER_SIZE_CD'] == '40'))]
data = data[(data['CDET_Days'] > 0) & (data['Std_FreeTimeGranted'] > 0) & (data['CONSIGNEE_CUSTOMER_CD'] != 'NULL')]
# data.display()

/databricks/spark/python/pyspark/sql/pandas/utils.py:105: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [BOOKED_FFE] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
  warnings.warn(


In [ ]:
unique_dipla = list(data['DIPLA_CITY_CD'].unique())
unique_dipla.sort()
unique_spot_yn = list(data['Spot_YN'].unique())
unique_container_size = list(data['CONTAINER_SIZE_CD'].unique())
combined_list = [unique_dipla, unique_spot_yn, unique_container_size]
combinations = list(itertools.product(*combined_list))
print(len(combinations))
# combinations

2196


In [ ]:
combination = ['INJHT', 'Y', '20']
df = data[(data['DIPLA_CITY_CD']==combination[0])&(data['Spot_YN']==combination[1])&(data['CONTAINER_SIZE_CD']==combination[2])]
df = df[['DIPLA_CITY_CD', 'Spot_YN', 'CONTAINER_SIZE_CD', 'CONSIGNEE_CUSTOMER_CD', 'CommoditySubType_Dsc', 'CDET_Days', 'PRICE_CALC_DT']]
df = df.groupby(['DIPLA_CITY_CD', 'Spot_YN', 'CONTAINER_SIZE_CD', 'CONSIGNEE_CUSTOMER_CD', 'CommoditySubType_Dsc','PRICE_CALC_DT'], as_index=False).mean('CDET_Days')
df['CDET_Days'] = df['CDET_Days'].round()
df = df.sort_values(['DIPLA_CITY_CD', 'Spot_YN', 'CONTAINER_SIZE_CD', 'CONSIGNEE_CUSTOMER_CD', 'CommoditySubType_Dsc', 'PRICE_CALC_DT'])
# temp = df[df['CONSIGNEE_CUSTOMER_CD']=='41000036042']
# temp
df.head()

,DIPLA_CITY_CD,Spot_YN,CONTAINER_SIZE_CD,CONSIGNEE_CUSTOMER_CD,CommoditySubType_Dsc,PRICE_CALC_DT,CDET_Days
0,INJHT,Y,20,10200188275,Metal,2021-09-10,5.0
1,INJHT,Y,20,10900018059,Metal,2022-02-10,6.0
2,INJHT,Y,20,11800123410,Personal care and cleaning products,2022-04-28,6.0
3,INJHT,Y,20,11801030274,Chemicals,2022-07-01,14.0
4,INJHT,Y,20,12200233241,Metal,2021-08-24,11.0


In [ ]:
df_rank = df.loc[df.groupby(['DIPLA_CITY_CD', 'CONSIGNEE_CUSTOMER_CD']).PRICE_CALC_DT.idxmin()]
df_rank.head()

,DIPLA_CITY_CD,Spot_YN,CONTAINER_SIZE_CD,CONSIGNEE_CUSTOMER_CD,CommoditySubType_Dsc,PRICE_CALC_DT,CDET_Days
0,INJHT,Y,20,10200188275,Metal,2021-09-10,5.0
1,INJHT,Y,20,10900018059,Metal,2022-02-10,6.0
2,INJHT,Y,20,11800123410,Personal care and cleaning products,2022-04-28,6.0
3,INJHT,Y,20,11801030274,Chemicals,2022-07-01,14.0
4,INJHT,Y,20,12200233241,Metal,2021-08-24,11.0


In [ ]:
# For New Customers if historical data of commodity is present
master_tt_df = pd.DataFrame()
for combination in combinations:
  df = data[(data['DIPLA_CITY_CD']==combination[0])&(data['Spot_YN']==combination[1])&(data['CONTAINER_SIZE_CD']==combination[2])]
  df = df[['DIPLA_CITY_CD', 'Spot_YN', 'CONTAINER_SIZE_CD', 'CONSIGNEE_CUSTOMER_CD', 'CommoditySubType_Dsc', 'CDET_Days', 'PRICE_CALC_DT']]
  df = df.groupby(['DIPLA_CITY_CD', 'Spot_YN', 'CONTAINER_SIZE_CD', 'CONSIGNEE_CUSTOMER_CD', 'CommoditySubType_Dsc','PRICE_CALC_DT'], as_index=False).mean('CDET_Days')
  df['CDET_Days'] = df['CDET_Days'].round()
  df_rank = df.loc[df.groupby(['DIPLA_CITY_CD', 'CONSIGNEE_CUSTOMER_CD']).PRICE_CALC_DT.idxmin()]
  df_rank = df_rank.sort_values(by=['CONSIGNEE_CUSTOMER_CD', 'CommoditySubType_Dsc', 'PRICE_CALC_DT'])
  df_mean = df_rank.groupby(['DIPLA_CITY_CD', 'Spot_YN', 'CONTAINER_SIZE_CD', 'CommoditySubType_Dsc'], as_index=False).mean('CDET_Days')
  master_tt_df = pd.concat([master_tt_df, df_mean], axis=0)

master_tt_df['CDET_Days'] = master_tt_df['CDET_Days'].round()
master_tt_df = master_tt_df.rename(columns={'CDET_Days':'Avg_CDET_Days'})
master_tt_df.reset_index(inplace=True, drop=True)
master_tt_df.head()

,DIPLA_CITY_CD,Spot_YN,CONTAINER_SIZE_CD,CommoditySubType_Dsc,Avg_CDET_Days
0,AEAMN,N,20,Appliances and kitchenware,9.0
1,AEAMN,N,20,Beverages,9.0
2,AEAMN,N,20,Chemicals,12.0
3,AEAMN,N,20,Dairy products,18.0
4,AEAMN,N,20,Foodstuff,10.0


In [ ]:
# For New Customers if historical data of commodity is NOT present
all_commodity_avg = pd.DataFrame()

df = master_tt_df[['DIPLA_CITY_CD', 'Spot_YN', 'CONTAINER_SIZE_CD', 'Avg_CDET_Days']]
df = df.sort_values(by=['DIPLA_CITY_CD', 'Spot_YN', 'CONTAINER_SIZE_CD'])
all_commodity_avg = df.groupby(['DIPLA_CITY_CD', 'Spot_YN', 'CONTAINER_SIZE_CD'], as_index=False).mean('Avg_CDET_Days')

all_commodity_avg['Avg_CDET_Days'] = all_commodity_avg['Avg_CDET_Days'].round()
all_commodity_avg['CommoditySubType_Dsc'] = 'Others'
all_commodity_avg.reset_index(inplace=True, drop=True)

master_tt_df = pd.concat([master_tt_df, all_commodity_avg], axis=0)
master_tt_df = master_tt_df.sort_values(['DIPLA_CITY_CD', 'Spot_YN', 'CONTAINER_SIZE_CD'])
master_tt_df = master_tt_df.rename(columns={'DIPLA_CITY_CD':'DIPLA_CD', 'CONTAINER_SIZE_CD':'Cont_Size', 'CommoditySubType_Dsc':'Commodity', 'Avg_CDET_Days':'TT_Pred'})
master_tt_df = master_tt_df.astype({'TT_Pred':np.int8})
master_tt_df['Cont_Type'] = 'Dry'
master_tt_df = master_tt_df[['DIPLA_CD', 'Spot_YN', 'Cont_Type', 'Cont_Size', 'Commodity', 'TT_Pred']]
master_tt_df.reset_index(inplace=True, drop=True)
master_tt_df.head()

,DIPLA_CD,Spot_YN,Cont_Type,Cont_Size,Commodity,TT_Pred
0,AEAMN,N,Dry,20,Appliances and kitchenware,9
1,AEAMN,N,Dry,20,Beverages,9
2,AEAMN,N,Dry,20,Chemicals,12
3,AEAMN,N,Dry,20,Dairy products,18
4,AEAMN,N,Dry,20,Foodstuff,10


In [ ]:
master_tt_df_spark = spark.createDataFrame(master_tt_df)
master_tt_df_spark.write.mode("overwrite").saveAsTable("dnd.master_tt_df_v2")

In [ ]:
new_consignee = spark.sql(""" select * from dnd.master_tt_df_v2 """)
new_consignee = new_consignee.toPandas()
new_consignee.head()

,DIPLA_CD,Spot_YN,Cont_Type,Cont_Size,Commodity,TT_Pred
0,ARMDP,Y,Dry,40,Others,7
1,ARPDO,N,Dry,20,Chemicals,4
2,ARPDO,N,Dry,20,Others,4
3,ARPDO,N,Dry,40,Plastic and rubber,6
4,ARPDO,N,Dry,40,Others,6


In [ ]:
commodity =spark.read.format("delta").load("dbfs:/mnt/Gen2_prod_cleansed_gcss/mars_commodities/")
commodity.createOrReplaceTempView("commodity")
commodity = spark.sql(""" select distinct commodity.code, name from commodity  order by 2""")

commodity = commodity.toPandas()
commodity = commodity.rename(columns={'name':'Commodity', 'code':'Commodity_CD'})
commodity = commodity[commodity['Commodity_CD'].apply(lambda x:len(x) == 4)]
commodity = commodity.drop_duplicates()

new_consignee = new_consignee.merge(commodity, on = 'Commodity', how='left')
new_consignee['Operator'] = 'Maersk'

new_consignee.tail()

,DIPLA_CD,Spot_YN,Cont_Type,Cont_Size,Commodity,TT_Pred,Commodity_CD,Operator
43988,MACAS,Y,Dry,20,Appliances and kitchenware,15,0014,Maersk
43989,MACAS,Y,Dry,20,Beverages,22,0002,Maersk
43990,MACAS,Y,Dry,20,Bone and meal,18,0003,Maersk
43991,MACAS,Y,Dry,20,Chemicals,11,0016,Maersk
43992,MACAS,Y,Dry,20,Dairy products,34,0004,Maersk


In [ ]:
new_consignee['Default'] = 'N'

new_consignee1 = new_consignee.copy()
new_consignee1 = new_consignee1[['DIPLA_CD', 'TT_Pred']].groupby(['DIPLA_CD'], as_index = False).mean()

new_consignee2 = new_consignee1.copy()
new_consignee2['DIPLA_CD'] = 'Others'
new_consignee2 = new_consignee2.groupby(['DIPLA_CD'], as_index = False).mean()

new_consignee1 = new_consignee1.append(new_consignee2)
new_consignee1['TT_Pred'] = round(new_consignee1['TT_Pred'])
new_consignee1['Default'] = 'Y'
new_consignee = new_consignee.append(new_consignee1)

new_consignee[[ 'Cont_Size', 'TT_Pred']] = new_consignee[[ 'Cont_Size', 'TT_Pred']].fillna(0)#.reset_index().drop('index', axis=1)
new_consignee = new_consignee.fillna('')

new_consignee[['Cont_Size', 'TT_Pred']] = new_consignee[['Cont_Size', 'TT_Pred']].astype(int)

new_consignee['Operator'] = 'Maersk'
new_consignee = new_consignee[['Operator', 'DIPLA_CD', 'Spot_YN', 'Cont_Type', 'Cont_Size','Commodity', 'Commodity_CD', 'TT_Pred', 'Default']]
new_consignee.loc[new_consignee['Commodity']=='Others', 'Commodity_CD'] = 'Others'
new_consignee = new_consignee.reset_index().drop('index', axis=1)

new_consignee.loc[new_consignee['Operator']=='Maersk', 'Operator'] = 'MAEU'
new_consignee.tail()

/root/.ipykernel/5397/command-3488349057168502-804410216:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_consignee1 = new_consignee1.append(new_consignee2)
/root/.ipykernel/5397/command-3488349057168502-804410216:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_consignee = new_consignee.append(new_consignee1)


,Operator,DIPLA_CD,Spot_YN,Cont_Type,Cont_Size,Commodity,Commodity_CD,TT_Pred,Default
44538,MAEU,ZACPT,,,0,,,3,Y
44539,MAEU,ZADUR,,,0,,,7,Y
44540,MAEU,ZAEBS,,,0,,,7,Y
44541,MAEU,ZAPLZ,,,0,,,3,Y
44542,MAEU,Others,,,0,,,10,Y


In [ ]:
new_consignee[(new_consignee['DIPLA_CD'] == 'AEJAL') & (new_consignee['Default'] == 'Y')]

,Operator,DIPLA_CD,Spot_YN,Cont_Type,Cont_Size,Commodity,Commodity_CD,TT_Pred,Default
43995,MAEU,AEJAL,,,0,,,8,Y


In [ ]:
new_consignee[(new_consignee['DIPLA_CD'] == 'AEJAL') & (new_consignee['Default'] == 'Y')]

,Operator,DIPLA_CD,Spot_YN,Cont_Type,Cont_Size,Commodity,Commodity_CD,TT_Pred,Default
43995,MAEU,AEJAL,,,0,,,8,Y


In [ ]:
len(new_consignee)

44543

In [ ]:
new_consignee = spark.createDataFrame(new_consignee)
display(new_consignee)

Operator,DIPLA_CD,Spot_YN,Cont_Type,Cont_Size,Commodity,Commodity_CD,TT_Pred,Default
MAEU,ARMDP,Y,Dry,40,Others,Others,7,N
MAEU,ARPDO,N,Dry,20,Chemicals,0016,4,N
MAEU,ARPDO,N,Dry,20,Others,Others,4,N
MAEU,ARPDO,N,Dry,40,Plastic and rubber,0032,6,N
MAEU,ARPDO,N,Dry,40,Others,Others,6,N
MAEU,ARPDO,Y,Dry,20,Appliances and kitchenware,0014,16,N
MAEU,ARPDO,Y,Dry,20,Metal,0023,28,N
MAEU,ARPDO,Y,Dry,20,Plastic and rubber,0032,6,N
MAEU,ARPDO,Y,Dry,20,Textiles and apparel,0041,16,N
MAEU,ARPDO,Y,Dry,20,Others,Others,16,N


In [ ]:
new_consignee.count()

41099

In [ ]:
sqlschemaName= 'ctpprod.' 

dwDatabase = 'cftex_pricing'
  
dwServer =  'sqlserveridadndprod'
  
dwJdbcPort =  '1433'
  
dwJdbcExtraOptions =  'encrypt=true;trustServerCertificate=true;hostNameInCertificate=*.database.windows.net;loginTimeout=30;'


In [ ]:
dwUser = '****'
dwpass = '****'
dwJdbcExtraOptions = "encrypt=true;trustServerCertificate=true;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"
sqlDwUrlSmall = "jdbc:sqlserver://" + dwServer + ".database.windows.net:" + dwJdbcPort + ";database=" + dwDatabase + ";user=" + dwUser+";password=" + dwpass
sqlDwhTbl="newcng"

In [ ]:
from  pyspark.sql.functions import col,when
from pyspark.sql.types import DoubleType
from pyspark.sql.types import BooleanType
from pyspark.sql.types import IntegerType
new_consignee = new_consignee.withColumn("Cont_Size", new_consignee["Cont_Size"].cast(IntegerType())).withColumn("TT_Pred", new_consignee["TT_Pred"].cast(IntegerType()))

In [ ]:
new_consignee.write.format("jdbc").mode("overwrite").option("truncate","true").option("BEST_EFFORT","true").option("tabLock","true").option("url", sqlDwUrlSmall).option("dbtable", sqlschemaName+sqlDwhTbl).option("user", dwUser).option("password", dwpass).option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").save()